# Downloading Colournet dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cp forest.zip gdrive/'My Drive'/Data/ 

cp: cannot stat 'forest.zip': No such file or directory


In [0]:
!wget http://cvcl.mit.edu/scenedatabase/forest.zip

--2019-08-15 13:53:42--  http://cvcl.mit.edu/scenedatabase/forest.zip
Resolving cvcl.mit.edu (cvcl.mit.edu)... 18.18.93.60
Connecting to cvcl.mit.edu (cvcl.mit.edu)|18.18.93.60|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23780317 (23M) [application/zip]
Saving to: ‘forest.zip’

forest.zip          100%[===================>]  22.68M   752KB/s    in 31s     

2019-08-15 13:54:15 (738 KB/s) - ‘forest.zip’ saved [23780317/23780317]



In [0]:
!7z x *.zip -o*


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 23780317 bytes (23 MiB)

Extracting archive: forest.zip
--
Path = forest.zip
Type = zip
Physical Size = 23780317

  0%     57% 236 - forest/natu158.jpg                             Everything is Ok

Folders: 1
Files: 329
Size:       24396250
Compressed: 23780317


# Preparing Libraries

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import cv2

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping
from keras.layers import *
from keras.models import Model, Sequential, load_model
from keras.regularizers import *
from keras.activations import *
from keras.optimizers import Adam
from keras.utils import Sequence
import math
from PIL import Image
from scipy import ndimage
import skimage.io as io
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import rescale, resize
import os

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input

Using TensorFlow backend.


# Custom Functions

In [0]:
def show(img): 
  plt.imshow(img, cmap = 'gray')
  
def show_predicted(l, ab):
  img = np.concatenate((l, ab), axis = 2)
  plt.imshow(lab2rgb(img))
  plt.pause(0.001)

# Data Loading

In [0]:
DS_PATH1 = '/content/forest/forest/' # ADD path/to/dataset
# Get images
train_data = []
for filename in os.listdir(DS_PATH1):
    if os.path.isfile(os.path.join(DS_PATH1, filename)):
        train_data.append(os.path.join(DS_PATH1, filename))

In [0]:
n = len(train_data)
X_train_main = np.random.choice(train_data, int(n*0.9))
X_val_main = np.array([x for x in train_data if x not in X_train_main])

In [0]:
print(n)

329


# Model

In [0]:
def build_generator():
        """
        Returns generator as Keras model.
        """
        g_input = Input(shape=(256, 256, 1))
        #128 x 128
        conv1 = Conv2D(64, (3, 3), padding='same', strides=2)(g_input)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation('relu')(conv1)

        conv2 = Conv2D(128, (3, 3), padding='same', strides=1)(conv1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation('relu')(conv2)

        #64 x 64
        conv3 = Conv2D(128, (3, 3), padding='same', strides=2)(conv2)
        conv3 = BatchNormalization()(conv3)
        conv3 = Activation('relu')(conv3)

        conv4 = Conv2D(256, (3, 3), padding='same', strides=1)(conv3)
        conv4 = BatchNormalization()(conv4)
        conv4 = Activation('relu')(conv4)

        #32 x 32
        conv5 = Conv2D(512, (3, 3), padding='same', strides=2)(conv4)
        conv5 = BatchNormalization()(conv5)
        conv5 = Activation('relu')(conv5)

        #64 x 64
        conv6 = UpSampling2D(size=(2, 2))(conv5)
        conv6 = Conv2D(256, (3, 3), padding='same')(conv6)
        conv6 = BatchNormalization()(conv6)
        conv6 = Activation('relu')(conv6)
        conv6 = Concatenate(axis=-1)([conv6,conv4])

        conv7 = Conv2D(256, (3, 3), padding='same')(conv6)
        conv7 = BatchNormalization()(conv7)
        conv7 = Activation('relu')(conv7)

        #128 x 128
        up2 = UpSampling2D(size=(2, 2))(conv7)
        conv8 = Conv2D(128, (3,3), padding='same')(up2)
        conv8 = BatchNormalization()(conv8)
        conv8 = Activation('relu')(conv8)
        conv8 = Concatenate(axis=-1)([conv8,conv2])

        conv9 = Conv2D(128, (3, 3), padding='same')(conv8)
        conv9 = BatchNormalization()(conv9)
        conv9 = Activation('relu')(conv9)

        up3 = UpSampling2D(size=(2, 2))(conv9)
        conv10 = Conv2D(64, (3,3), padding='same')(up3)
        conv10 = BatchNormalization()(conv10)
        conv10 = Activation('relu')(conv10)

        conv11 = Conv2D(2, (3, 3), padding='same')(conv10)
        conv11 = Activation('tanh')(conv11)

        model = Model(inputs=g_input,outputs=conv11)
        return model

In [0]:
def build_discriminator():
        """
        Returns discriminator as Keras model.
        """
        model = Sequential()
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=(256,256,2), strides=2))
        model.add(LeakyReLU(.2))

        model.add(AveragePooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), padding='same',strides=1))
        model.add(LeakyReLU(.2))
        model.add(Dropout(.25))

        model.add(AveragePooling2D(pool_size=(2, 2)))
        model.add(Conv2D(128, (3, 3), padding='same',strides=1))
        model.add(LeakyReLU(.2))
        model.add(Dropout(.25))

        model.add(AveragePooling2D(pool_size=(2, 2)))
        model.add(Conv2D(256, (3, 3), padding='same',strides=2))
        model.add(LeakyReLU(.2))
        model.add(Dropout(.25))

        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        return model


In [0]:
import keras.backend as K

def cus_acc(y_true, y_pred):
  
  x = K.cast(K.not_equal(y_pred,0),tf.float32)
  y = K.equal(x, y_true)
  return K.mean(y)

## Generator

In [0]:
generator = build_generator()
generator.compile(loss='mse', optimizer=Adam(lr=.001))
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 128, 128, 64) 640         input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_41 (BatchNo (None, 128, 128, 64) 256         conv2d_61[0][0]                  
__________________________________________________________________________________________________
activation_49 (Activation)      (None, 128, 128, 64) 0           batch_normalization_41[0][0]     
__________________________________________________________________________________________________
conv2d_62 

## Discriminator

In [0]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(lr=.0001),
                      metrics=['accuracy'])
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_72 (Conv2D)           (None, 128, 128, 32)      608       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 128, 128, 32)      0         
_________________________________________________________________
average_pooling2d_13 (Averag (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 64, 64, 64)        0         
_________________________________________________________________
average_pooling2d_14 (Averag (None, 32, 32, 64)        0         
__________

## GAN

In [0]:
gan_input = Input(shape=(256,256,1))
img_color = generator(gan_input)
discriminator.trainable = False
real_or_fake = discriminator(img_color)
gan = Model(gan_input,real_or_fake)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=.001,decay=1e-5), metrics = ['accuracy'])

In [0]:
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 256, 256, 1)       0         
_________________________________________________________________
model_10 (Model)             (None, 256, 256, 2)       4729922   
_________________________________________________________________
sequential_5 (Sequential)    (None, 1)                 404513    
Total params: 5,134,435
Trainable params: 4,726,082
Non-trainable params: 408,353
_________________________________________________________________


# Data Generator

In [0]:
class dis_gen(Sequence):
  
  def __init__(self, X, batch_size = 32):
    self.img_loc = X
    self.batch_size = batch_size
    self.size = len(self.img_loc)
    
  def __getitem__(self, idx):
    start = idx*self.batch_size
    end = min((idx+1)*self.batch_size, self.size)
    batch = np.empty(((end-start),256,256,3))
    j=0
    '''
    exception handling while reading image
    '''
    for i in self.img_loc[start:end]:
      try:
        batch[j,...] = cv2.imread(i)[...,::-1] # RBG image 
      except:
        pass
      j+=1
      
    batch /= 255.0
    gray_batch = gray2rgb(rgb2gray(batch))
    lab_batch = rgb2lab(batch)
    X_train_L = lab_batch[...,0]
    X_train_L = X_train_L[...,None]
    X_train_AB = lab_batch[:,:,:,1:]/128.0
    generated_images = generator.predict(X_train_L)
    X_train = np.concatenate((X_train_AB, generated_images))
    n = len(X_train_L)
    y_train = np.array([[1]] * n + [[0]] * n)
    rand_arr = np.arange(len(X_train))
    np.random.shuffle(rand_arr)
    X_train = X_train[rand_arr]
    y_train = y_train[rand_arr]
    return (X_train, y_train)
      
  def __len__(self):
    return math.ceil(self.size / self.batch_size)

In [0]:
class gan_gen(Sequence):
  def __init__(self, X, batch_size = 16):
    self.img_loc = X
    self.batch_size = batch_size
    self.size = len(self.img_loc)
  
  def __getitem__(self, idx):
    start = idx*self.batch_size
    end = min((idx+1)*self.batch_size, self.size)
    batch = np.empty(((end-start),256,256,3))
    j=0
    '''
    exception handling while reading image
    '''
    for i in self.img_loc[start:end]:
      try:
        batch[j,...] = cv2.imread(i)[...,::-1] # RBG image 
      except:
        pass
      j+=1
    batch /= 255.0
    gray_batch = gray2rgb(rgb2gray(batch))
    lab_batch = rgb2lab(batch)
    X_train_L = lab_batch[...,0]
    X_train_L = X_train_L[...,None]
    n = len(X_train_L)
    y_train = np.ones([n,1])
    return (X_train_L, y_train)
  
  def __len__(self):
    return math.ceil(self.size / self.batch_size)

# Train

In [0]:
batch_size = 32
def train( X_train,  X_test, epochs):
  acc_dis = 0
  n = len(X_train)
  for e in range(epochs):
    print('.......Evaluating discriminator.......')
    disgen = dis_gen(X_train, batch_size)
    valgen = dis_gen(X_test, batch_size)
    metric = discriminator.evaluate_generator(valgen)
    acc_dis = metric[1]
    print('Accuracy : %f' %(metric[1]))
    print('...........training discriminator.........')
    if(e%3==2):
       noise = np.random.rand(n,256,256,2) * 2 -1
       discriminator.fit(noise, np.zeros([n,1]), 32, epochs=1)
    while(acc_dis <= 0.89):
      discriminator.fit_generator(disgen, epochs=1)
      metric = discriminator.evaluate_generator(valgen)
      acc_dis = metric[1]
      print('Accuracy : %f' %(metric[1]))
    print('............training gan............')
    #while(acc_dis>0.81):
    gan.fit_generator(gan_gen(X_train), epochs=1)
    metric = discriminator.evaluate_generator(valgen)
    test()
    acc_dis = metric[1]
    print(acc_dis)
    #test()
    #test()
    print(e + 1,"batches done")

In [0]:
dis_gen(X_train_main).__getitem__(0)

In [0]:
train(X_train_main, X_val_main, 25)

# Saving and loading weights

In [0]:
cd ~/../content/gdrive/'My Drive'/Data/

/content/gdrive/My Drive/Data


In [0]:
gan.save('gan.h5')

In [0]:
discriminator.save_weights('dis_weights1_1280.h5')

In [0]:
generator.save_weights('gen_weights1_1280.h5')

In [0]:
gan.save_weights('gan_weights1_1280.h5')

In [0]:
cd ~/../content/

/content


# Colourization

In [0]:
original = plt.imread(X_train_main[5])/255
gray = gray2rgb(rgb2gray(original))

In [0]:
show(original)

In [0]:
lab = rgb2lab(gray)
lab2 = rgb2lab(original)

In [0]:
predicted_ab = generator.predict((lab2[...,0]).reshape((1,256,256,1)))
ab = predicted_ab.reshape(256,256,2)
show_predicted((lab2[...,0]).reshape((256,256,1)),ab*128)

In [0]:
def test():
  original = plt.imread(X_train_main[3])/255
  gray = gray2rgb(rgb2gray(original))
  lab = rgb2lab(gray)
  lab2 = rgb2lab(original)
  predicted_ab = generator.predict((lab2[...,0]).reshape((1,256,256,1)))
  ab = predicted_ab.reshape(256,256,2)
  show_predicted((lab2[...,0]).reshape((256,256,1)),ab*128)

In [0]:
test()